In [1]:
import sys

sys.path.append("/home/jupyter-aamir09/mlops2_with_dagster/")


from mlops2_with_dagster import encoder_pipeline, features_pipeline
from pathlib import Path
from joblib import dump, load
from hamilton import driver, base
from mlops2_with_dagster.utils import get_project_dir, printse

from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)
from ortho.utils.logger import Logger
from ortho.ortho.decorators.task import task
from typing import Tuple

import pandas as pd 
import mlflow as mf
from ortho.ortho.callbacks.mlflow import MlFlowCallBack
from ortho.ortho.callbacks.logger import LoggerCallBack
import os


logger = Logger().logger()

   

        
        

None


2023-08-30 13:10:22.473534: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 13:10:22.505871: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-30 13:10:22.506667: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 13:10:22,977 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-08-30 13:10:23.029334: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-30 13:10:23,154 - DEBUG - Creating converter from 7 to 5
2023-08-30 13:10:23,155 - DEBUG - Creating converter from 

In [2]:
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/jupyter-aamir09/mlops2_with_dagster


In [3]:
train_data : Path = project_dir/"data/train.csv"
test_data : Path = project_dir/"data/test.csv"
encoder_file: Path = project_dir/"warehouse/encoders.joblib"
data: Path = project_dir/"data/train.csv"

In [4]:
# parameters
import dagstermill
df = pd.read_csv(data)
encoders = load(encoder_file)
context = dagstermill.get_context(op_config={'datatype': 'xxxx'})


2023-08-30 13:10:23,722 - INFO - Context impl SQLiteImpl.
2023-08-30 13:10:23,723 - INFO - Will assume non-transactional DDL.
2023-08-30 13:10:23,735 - INFO - Running stamp_revision  -> 5771160a95ad
2023-08-30 13:10:23,735 - DEBUG - new branch insert 5771160a95ad
2023-08-30 13:10:23,749 - INFO - Context impl SQLiteImpl.
2023-08-30 13:10:23,750 - INFO - Will assume non-transactional DDL.
2023-08-30 13:10:23,760 - INFO - Running stamp_revision  -> 5771160a95ad
2023-08-30 13:10:23,760 - DEBUG - new branch insert 5771160a95ad


In [5]:
class Transforms:
    
    def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
        self.load_from_artifact = load_from_artifact
        self.callbacks = callbacks
        self.experiment_name = experiment_name
        self.run_name = run_name
        os.environ["MLFLOW_EXPERIMENT_NAME"] = self.experiment_name
        os.environ["MLFLOW_RUN_NAME"] = self.run_name
        
        self.index_col = 'passengerid'
        self.target_col = "survived"
        self.cat_cols = ["sex", "cabin", "embarked"]
        self.config = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'categorical_columns': self.cat_cols
        }
    
    @task(build_on_previous_run=True, end_mlflow_run=False)       
    def transform(self, df_train, encoders=None):
        transform_dr = driver.Driver(self.config, encoder_pipeline, features_pipeline)
        ddf = dict(df = df_train, **encoders['encoders'])
        output_nodes = ['final_imputed_features']
        
        output = transform_dr.execute(output_nodes, inputs = ddf)
        
        data = {"transformed_data": output }
        payload = {"artifact_path": "/home/jupyter-aamir09/mlops2_with_dagster/artifacts/mlflow_artfacts"}
        
        return data, payload
    
    def run(self, *args, **kwargs):
        
        return self.transform(*args, **kwargs)
        
        
    

In [7]:
import dagstermill
transformer = Transforms(experiment_name="Mlflow_with_Dagster",
                          run_name="BadamBhum",
                          callbacks = [LoggerCallBack("/home/jupyter-aamir09/mlops2_with_dagster/notebooks", kernel_names=["transform"]),
                                       MlFlowCallBack(kernel_names=["transform"])])

# mf.end_run()
dagstermill.yield_result(transformer.run(df_train=df, encoders=encoders)[0]["transformed_data"], output_name="transformed_data")

2023-08-30 13:13:48,162 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 13:13:48,168 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=Mlflow_with_Dagster HTTP/1.1" 200 241
2023-08-30 13:13:48,170 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 13:13:48,177 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=1f0a6e058efb4c5d9e9fbecc36928d78&run_id=1f0a6e058efb4c5d9e9fbecc36928d78 HTTP/1.1" 200 1260
2023-08-30 13:13:48,179 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 13:13:48,184 - DEBUG - http://127.0.0.1:5002 "POST /api/2.0/mlflow/runs/update HTTP/1.1" 200 423
2023-08-30 13:13:48,186 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 13:13:48,191 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/runs/get?run_uuid=1f0a6e058efb4c5d9e9fbecc36928d78&run_id=1f0a6e058efb4c5d9e9fbecc36928d78 HTTP/1.1" 200 1260
2023-08-30 13:13:48,192 - INFO - Successfuly initiated run with nam

['self', 'df_train', 'encoders']


2023-08-30 13:13:52,208 - DEBUG - Retry: /api/2.0/mlflow-artifacts/artifacts/1/1f0a6e058efb4c5d9e9fbecc36928d78/artifacts/df_train.pickle
2023-08-30 13:13:52,210 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 13:13:52,215 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow-artifacts/artifacts/1/1f0a6e058efb4c5d9e9fbecc36928d78/artifacts/df_train.pickle HTTP/1.1" 500 258
2023-08-30 13:13:52,216 - DEBUG - Incremented Retry for (url='/api/2.0/mlflow-artifacts/artifacts/1/1f0a6e058efb4c5d9e9fbecc36928d78/artifacts/df_train.pickle'): Retry(total=2, connect=5, read=5, redirect=5, status=2)
2023-08-30 13:14:00,226 - DEBUG - Retry: /api/2.0/mlflow-artifacts/artifacts/1/1f0a6e058efb4c5d9e9fbecc36928d78/artifacts/df_train.pickle
2023-08-30 13:14:00,228 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-08-30 13:14:00,233 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow-artifacts/artifacts/1/1f0a6e058efb4c5d9e9fbecc36928d78/artifacts/df_train.pickle HTTP/1.1" 500 258
2023-0

/tmp/tmptyislrj1/encoders.pickle
{'encoders': {'encoders': {'cabinencoder': LabelEncoder(), 'sexencoder': LabelEncoder(), 'embarkedencoder': LabelEncoder()}}}
<class 'pandas.core.frame.DataFrame'>


2023-08-30 13:14:48,515 - DEBUG - http://127.0.0.1:5002 "PUT /api/2.0/mlflow-artifacts/artifacts/1/1f0a6e058efb4c5d9e9fbecc36928d78/artifacts/encoders.pickle HTTP/1.1" 200 2


,pclass,age,fare,cabin_category,sex_category,embarked_category,family
passengerid,,,,,,,
0,1,0.00,27.14,2,1,2,2
1,3,0.00,13.35,8,1,2,0
2,3,0.33,71.29,8,1,2,3
3,3,19.00,13.04,8,1,2,0
4,3,25.00,7.76,8,1,2,0
...,...,...,...,...,...,...,...
99995,2,62.00,14.86,3,0,0,0
99996,2,66.00,11.15,8,1,2,0
99997,3,37.00,9.95,8,1,2,0


2023-08-30 13:14:48,630 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event": "os_hamilton_run_start", "properties": {"os_type": "posix", "os_version": "Linux-5.4.0-124-generic-x86_64-with-glibc2.31", "python_version": "3.10.12/CPython", "distinct_id": "e4e272c1-5845-4b12-8bd8-09d84d585dbc", "hamilton_version": [1, 27, 2], "telemetry_version": "0.0.1", "number_of_nodes": 42, "number_of_modules": 2, "number_of_config_items": 3, "decorators_used": {"parameterize_sources": 4, "extract_columns": 2, "extract_fields": 1}, "graph_adapter_used": "hamilton.base.SimplePythonDataFrameGraphAdapter", "result_builder_used": "hamilton.base.PandasDataFrameResult", "driver_run_id": "88beb813-e398-4538-b488-766ef32a2c10", "error": null, "graph_executor_class": "DefaultGraphExecutor"}}'].
2023-08-30 13:14:48,710 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "e